![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb)

# 24. Improved Entity Resolvers in Spark NLP with sBert

In [ ]:
import json, os
from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import json
import os
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
import sparknlp_jsl
import sparknlp

from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.3.4
Spark NLP_JSL Version : 3.3.4


<h1>!!! Warning !!!</h1>

**If you get an error related to Java port not found 55, it is probably because that the Colab memory cannot handle the model and the Spark session died. In that case, try on a larger machine or restart the kernel at the top and then come back here and rerun.**

## ICD10CM pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

icd_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver])

icd_lp = LightPipeline(icd_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


## ICD10CM-HCC pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
hcc_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcc_resolver])

hcc_lp = LightPipeline(hcc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
Approximate size to download 1.4 GB
[OK!]


## RxNorm pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp = LightPipeline(rxnorm_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
Approximate size to download 714.5 MB
[OK!]


## NDC pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
ndc_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_ndc", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("ndc_code")\
      .setDistanceFunction("EUCLIDEAN")

ndc_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        ndc_resolver])

ndc_lp = LightPipeline(ndc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_ndc download started this may take some time.
Approximate size to download 391.4 MB
[OK!]


## CPT pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])

cpt_lp = LightPipeline(cpt_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_cpt_procedures_augmented download started this may take some time.
Approximate size to download 78.3 MB
[OK!]


## SNOMED pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings_aux_concepts","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

snomed_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver])

snomed_lp = LightPipeline(snomed_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_snomed_findings_aux_concepts download started this may take some time.
Approximate size to download 4.3 GB
[OK!]


## LOINC Pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
loinc_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_loinc_augmented", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("loinc_code")\
      .setDistanceFunction("EUCLIDEAN")

loinc_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        loinc_resolver])

loinc_lp = LightPipeline(loinc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_loinc_augmented download started this may take some time.
Approximate size to download 1.4 GB
[OK!]


## HCPCS Pipeline

In [4]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
hcpcs_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_hcpcs", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("hcpcs_code")\
      .setDistanceFunction("EUCLIDEAN")

hcpcs_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcpcs_resolver])

hcpcs_lp = LightPipeline(hcpcs_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_hcpcs download started this may take some time.
Approximate size to download 18.3 MB
[OK!]


## UMLS Pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
umls_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        umls_resolver])

umls_lp = LightPipeline(umls_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_umls_major_concepts download started this may take some time.
Approximate size to download 761 MB
[OK!]


## MeSH pipeline

In [5]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
mesh_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_mesh","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("mesh_code")\
      .setDistanceFunction("EUCLIDEAN")

mesh_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        mesh_resolver])

mesh_lp = LightPipeline(mesh_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_mesh download started this may take some time.
Approximate size to download 956.7 MB
[OK!]


## HPO Pipeline

In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
hpo_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_HPO", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

hpo_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hpo_resolver])

hpo_lp = LightPipeline(hpo_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_HPO download started this may take some time.
Approximate size to download 97.9 MB
[OK!]


## All the resolvers in the same pipeline 
### (just to show how it is done..  will not be used in this notebook)

In [ ]:

documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
snomed_ct_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_resolver = SentenceEntityResolverModel.pretrained("sbert_biobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

loinc_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_loinc_augmented", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("loinc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcpcs_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_hcpcs", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("hcpcs_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

hpo_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_HPO", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

mesh_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_mesh","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("mesh_code")\
      .setDistanceFunction("EUCLIDEAN")

ndc_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_ndc", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("ndc_code")\
      .setDistanceFunction("EUCLIDEAN")


resolver_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver,
        hcc_resolver,
        rxnorm_resolver,
        ndc_resolver,
        cpt_resolver,
        snomed_ct_resolver,
        loinc_resolver,
        hcpcs_resolver,
        umls_resolver,
        mesh_resolver
        hpo_resolver])

resolver_lp = LightPipeline(resolver_pipelineModel)


## Utility functions

In [6]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', hcc=False, aux_label=False):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        
        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
                
        elif aux_label:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes,'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    if hcc:

        df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
        df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
        df['hcc_score'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])
    
    elif aux_label:

        df['gt'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[0] for i in x])
        df['concept'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[1] for i in x])
        df['aux'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[2] for i in x])

    df = df.drop(['all_k_aux_labels'], axis=1)
    
    return df



## Getting some predictions from resolvers

In [ ]:
text = 'bladder cancer'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 66.2 ms, sys: 18.6 ms, total: 84.8 ms
Wall time: 7.86 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]"


In [ ]:
text = 'severe stomach pain'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 19.2 ms, sys: 2.9 ms, total: 22.1 ms
Wall time: 1.06 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,severe stomach pain,0,18,R101,"[R101, R109, R1082, M791, R104, R1084, K388, R529, F454, R079, R53, R1013, R52, M7918, R51, R520, R4589, R1033, R072, R103, R1010]","[upper abdominal pain, intractable abdominal pain, epigastric pain, sore pain, generalized colicky abdominal pain, generalized abdominal pain, colicky abdominal pain, severe pain, psychosomatic abdominal pain, upper chest pain, exhausting with pain, burning epigastric pain, heavy pain, abdominal muscle pain, head pain, horrible present pain, cruel with pain, central abdominal pain, esophageal pain, o/e - epigastric pain, ulcer-type pain]","[0.0523, 0.0551, 0.0565, 0.0570, 0.0578, 0.0589, 0.0605, 0.0613, 0.0634, 0.0641, 0.0644, 0.0664, 0.0656, 0.0663, 0.0677, 0.0682, 0.0686, 0.0711, 0.0724, 0.0721, 0.0724]"


In [ ]:
text = 'bladder cancer'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 83.7 ms, sys: 7.91 ms, total: 91.6 ms
Wall time: 7.7 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,bladder cancer,0,13,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 1, 1, 1, 1]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


In [ ]:
text = 'severe stomach pain'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 23.3 ms, sys: 3.95 ms, total: 27.3 ms
Wall time: 1.34 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,severe stomach pain,0,18,R101,"[R101, R109, R1082, M791, R104, R1084, K388, R529, F454, R079, R53, R1013, R52, M7918, R51, R520, R4589, R1033, R072, R103, R1010]","[upper abdominal pain, intractable abdominal pain, epigastric pain, sore pain, generalized colicky abdominal pain, generalized abdominal pain, colicky abdominal pain, severe pain, psychosomatic abdominal pain, upper chest pain, exhausting with pain, burning epigastric pain, heavy pain, abdominal muscle pain, head pain, horrible present pain, cruel with pain, central abdominal pain, esophageal pain, o/e - epigastric pain, ulcer-type pain]","[0.0523, 0.0551, 0.0565, 0.0570, 0.0578, 0.0589, 0.0605, 0.0613, 0.0634, 0.0641, 0.0644, 0.0664, 0.0656, 0.0663, 0.0677, 0.0682, 0.0686, 0.0711, 0.0724, 0.0721, 0.0724]","[0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 26.6 ms, sys: 3.78 ms, total: 30.4 ms
Wall time: 1.69 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,861024,"[861024, 334738, 332848, 333262, 439563, 450523, 1744000, 541068, 402346, 1726496, 545206, 485246, 316350, 213092, 858858, 336846, 568808, 316844, 198370, 1946837, 451225, 882533, 856606, 1307630, 437723]","[metformin hydrochloride 100 mg/ml, fenofibrate 100 mg, ciprofibrate 100 mg, rutin 100 mg, fendiline 100 mg, perazine 100 mg, emtricitabine 100 mg, miglustat 100 mg, miglustat 100 mg, azacitidine 100 mg, azacitidine 100 mg, azacitidine 100 mg, niacin 100 mg, anzemet 100 mg, carnosine 100 mg, trimebutine 100 mg, torsemide 100 mg, torsemide 100 mg, torsemide 100 mg, abemaciclib 100 mg, pyrantel 100 mg, cleocin 100 mg, flumadine 100 mg, bosulif 100 mg, azintamide 100 mg]","[0.0658, 0.0684, 0.0697, 0.0765, 0.0775, 0.0804, 0.0818, 0.0811, 0.0811, 0.0812, 0.0812, 0.0812, 0.0811, 0.0836, 0.0828, 0.0844, 0.0846, 0.0846, 0.0846, 0.0840, 0.0844, 0.0849, 0.0850, 0.0843, 0.0851]"


In [ ]:
text = 'Advil Allergy Sinus'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 16.2 ms, sys: 1.81 ms, total: 18.1 ms
Wall time: 1.41 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Advil Allergy Sinus,0,18,215618,"[215618, 899469, 701329, 1311824, 702229, 1170796, 308151, 1170133, 1801249, 1170795, 1094448, 210238, 847500, 215138, 1373205, 543743, 847166, 876174, 899419, 2289242, 215127, 1170135, 1177219]","[benadryl allergy sinus, allergy hp, benadryl-d allergy and sinus, allergy d, elixsure allergy, benadryl-d allergy and sinus oral product, bhi allergy, benadryl allergy sinus oral product, aller-flo nasal product, benadryl-d allergy and sinus oral liquid product, scot-tussin allergy, sprayology allergease, allerx pe am, allergia-c, allergy complex, allergen ear, allerx df am, bactimicina allergy, allergy pollen, aller-cort nasal product, allercon, benadryl allergy sinus oral liquid product, scot-tussin allergy oral product]","[0.0715, 0.0818, 0.0953, 0.0936, 0.0990, 0.1022, 0.1042, 0.1098, 0.1115, 0.1133, 0.1126, 0.1204, 0.1182, 0.1185, 0.1222, 0.1224, 0.1227, 0.1227, 0.1218, 0.1280, 0.1266, 0.1292, 0.1309]"


In [ ]:
text = 'metformin 500 mg'

%time get_codes (ndc_lp, text, vocab='ndc_code')

CPU times: user 15.8 ms, sys: 3.39 ms, total: 19.2 ms
Wall time: 780 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 500 mg,0,15,70010006315,"[70010006315, 71052050750, 71052091050, 71052063005, 71052065655, 71052065905, 62207041613, 71052009700, 71052011005, 51662139201, 71052006505, 71052050850, 71052050701, 71052028005, 62207049147, 25000013498, 25000010197, 25000010198, 50222050347, 71052065605, 51552150806, 10631020602, 14474003601, 59898072002, 71052065850]","[metformin hydrochloride 500 mg/500mg, 500 mg in 1 drum , tablet, 5-fluorouracil 500 g/500g, 500 g in 1 container , powder, alpha-msh acetate 500 mg/500mg, 500 mg in 1 container , powder, ursodeoxycholic acid 500 g/500g, 500 g in 1 container , powder, acarbose 500 g/500g, 500 g in 1 container , powder, hydroxychloroquine sulfate 500 g/500g, 500 g in 1 container , powder, metformin hcl 500 mg/kg, 50 kg in 1 drum , powder, idebenone 500 g/500g, 500 g in 1 container , powder, orlistat 500 g/500g, 500 g in 1 container , powder, procainamide hci 500 mg/ml, 2 ml in 1 vial, multi-dose , injection, solution, zonisamide 500 g/500g, 500 g in 1 container , powder, dantrolene sodium 500 g/500g, 500 g in 1 container , powder, 5-fluorouracil 500 g/500g, 1000 g in 1 container , powder, furazolidone 500 g/500g, 500 g in 1 container , powder, metformin er 500 mg 500 mg/1, 500 tablet in 1 bottle , tablet, metformin hydrochloride 500 mg/1, 1 bag in 1 box , tablet, film coated, metformin hydrochloride 500 mg/1, 4 bag in 1 box , tablet, extended release, metformin hydrochloride 500 mg/1, 1 bag in 1 box , tablet, extended release, ingenol mebutate 500 ug/g, 2 tube in 1 carton , gel, acarbose 500 g/500g, 5000 g in 1 container , powder, metformin hcl 1 g/g, 500 g in 1 container , powder, metformin hydrochloride 500 mg/5ml, 473 ml in 1 bottle , solution, isoproterenol hydrochloride 500 g/500g, 500 g in 1 package , powder, bacitracin 500 [iu]/g, 144 packet in 1 box, unit-dose , ointment, lanthanum carbonate 500 g/500g, 500 g in 1 container , powder]","[0.2239, 0.2289, 0.2448, 0.2492, 0.2464, 0.2519, 0.2449, 0.2550, 0.2520, 0.2624, 0.2588, 0.2625, 0.2651, 0.2606, 0.2592, 0.2700, 0.2710, 0.2715, 0.2758, 0.2692, 0.2666, 0.2759, 0.2778, 0.2740, 0.2733]"


In [ ]:
text = 'aspirin 81 mg'

%time get_codes (ndc_lp, text, vocab='ndc_code')

CPU times: user 14.3 ms, sys: 3.02 ms, total: 17.3 ms
Wall time: 765 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,aspirin 81 mg,0,12,73089008114,"[73089008114, 71872708704, 71872715401, 68210101500, 69536028110, 63548086706, 71679001000, 68196090051, 00113400500, 69536018112, 73089008112, 63981056362, 63739043402, 63548086705, 00113046708, 70000017801, 41226048150, 69536028136, 50090483200, 70518273300, 11673070208, 52124001201, 69168031806, 69168031817, 17714000936]","[aspirin 81 mg/81mg, 81 mg in 1 carton , capsule, aspirin 81 mg 81 mg/1, 4 blister pack in 1 bag , tablet, aspirin 81 mg/1, 1 blister pack in 1 bag , tablet, coated, aspirin 81 mg/1, 1 bag in 1 drum , tablet, film coated, aspirin 81 mg/1, 3 bottle in 1 box , tablet, chewable, aspirin 81 mg/1, 36 bottle in 1 case , tablet, aspirin 81 mg/1, 1 bag in 1 drum , tablet, coated, aspirin 81 mg/1, 2 bottle in 1 package , tablet, coated, aspirin 81 mg/1, 1 bag in 1 drum , tablet, chewable, aspirin 81 mg/1, 1 bottle in 1 box , tablet, film coated, aspirin 81 mg/81mg, 1 blister pack in 1 carton , capsule, aspirin 81 mg/1, 1 bottle in 1 package , tablet, coated, aspirin 81 mg 81 mg/1, 30 blister pack in 1 box , tablet, aspirin 81 mg/1, 1 bag in 1 drum , tablet, aspirin 81 mg/1, 3 carton in 1 carton , tablet, chewable, aspirin 81 mg/1, 1 bottle in 1 carton , tablet, film coated, aspirin 81 mg/1, 1 bottle in 1 bottle , tablet, coated, aspirin 81 mg/1, 1 bottle in 1 box , tablet, chewable, aspirin 81 mg/1, 33 blister pack in 1 box, unit-dose , tablet, coated, aspirin 81 mg/1, 30 tablet in 1 blister pack , tablet, aspirin 81 mg/1, 3 bottle in 1 carton , tablet, chewable, aspirin 81 mg/1, 2 tablet, chewable in 1 package , tablet, chewable, aspirin 81 mg 81 mg/1, 1 bottle in 1 carton , tablet, aspirin 81 mg 81 mg/1, 300 tablet in 1 bottle , tablet, aspirin 81 mg/1, 36 carton in 1 bottle , tablet, chewable]","[0.1658, 0.1795, 0.1889, 0.1902, 0.1937, 0.2010, 0.2000, 0.2060, 0.2013, 0.2023, 0.2038, 0.2048, 0.2037, 0.2056, 0.2080, 0.2095, 0.2111, 0.2120, 0.2121, 0.2154, 0.2152, 0.2177, 0.2133, 0.2132, 0.2194]"


In [ ]:
text = 'heart surgery'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 17.1 ms, sys: 5.27 ms, total: 22.4 ms
Wall time: 241 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,heart surgery,0,12,33258,"[33258, 33999, 0306T, 0304T, 0302T, 93583, 1006057, 93566, 93565, 93568, 93564, 93567, 1013106, 33430, 33465, 1006147, 33425, 1014130]","[Cardiac surgery procedure [Operative tissue ablation and reconstruction of atria, performed at the time of other cardiac procedure(s), extensive (eg, maze procedure), without cardiopulmonary bypass (List separately in addition to code for primary procedure)], Cardiac surgery procedure [Unlisted procedure, cardiac surgery], Heart procedure [Interrogation device evaluation (in person) of intracardiac ischemia monitoring system with analysis, review, and report], Heart procedure [Insertion or removal and replacement of intracardiac ischemia monitoring system including imaging supervision and interpretation when performed and intra-operative interrogation and programming when performed; device only], Heart procedure [Insertion or removal and replacement of intracardiac ischemia monitoring system including imaging supervision and interpretation when performed and intra-operative interrogation and programming when performed; complete system (includes device and electrod], Operation on cardiac septum [Percutaneous transcatheter septal reduction therapy (eg, alcohol septal ablation) including temporary pacemaker insertion when performed], Surgical Procedures on the Heart and Pericardium [Surgical Procedures on the Heart and Pericardium], Insertion of catheter into heart chamber [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for selective right ventricular or right atrial angiography (List separately in addition to code for primary procedure)], Insertion of catheter into heart chamber [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for selective left ventricular or left atrial angiography (List separately in addition to code for primary procedure)], Insertion of catheter into heart chamber [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for pulmonary angiography (List separately in addition to code for primary procedure)], Insertion of catheter into heart chamber [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for selective opacification of aortocoronary venous or arterial bypass graft(s) (eg, aortocoronary saphenous vein, free radial artery, or free ma], Insertion of catheter into heart chamber [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for supravalvular aortography (List separately in addition to code for primary procedure)], Repair Procedures of Structural Heart Defect [Repair Procedures of Structural Heart Defect], Replacement, mitral valve, with cardiopulmonary bypass [Replacement, mitral valve, with cardiopulmonary bypass], Replacement, tricuspid valve, with cardiopulmonary bypass [Replacement, tricuspid valve, with cardiopulmonary bypass], Replacement, aortic valve, with cardiopulmonary bypass [Replacement, aortic valve, with cardiopulmonary bypass], Valvuloplasty, mitral valve, with cardiopulmonary bypass [Valvuloplasty, mitral valve, with cardiopulmonary bypass], Valvuloplasty, mitral valve, with cardiopulmonary bypass [Valvuloplasty, mitral valve, with cardiopulmonary bypass]]","[0.1031, 0.1031, 0.1377, 0.1377, 0.1377, 0.1645, 0.1845, 0.1904, 0.1904, 0.1904, 0.1904, 0.1904, 0.1949, 0.1971, 0.2066, 0.2047, 0.2107, 0.2107]"


In [ ]:
text = 'ct abdomen'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 8.44 ms, sys: 4.18 ms, total: 12.6 ms
Wall time: 216 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,ct abdomen,0,9,62284,"[62284, 74183, 74181, 76497, 49329, 70540, 49080, 49083, 70543, 33891, 34471, 34001, 1010526, 74175, 74160, 35180, 73206, 73202]","[CT of spine [Injection procedure for myelography and/or computed tomography, lumbar], MRI of abdomen [Magnetic resonance (eg, proton) imaging, abdomen; without contrast material(s), followed by with contrast material(s) and further sequences], MRI of abdomen [Magnetic resonance (eg, proton) imaging, abdomen; without contrast material(s)], CT Scan [Unlisted computed tomography procedure (eg, diagnostic, interventional)], Procedure on abdomen [Unlisted laparoscopy procedure, abdomen, peritoneum and omentum], MRI of neck [Magnetic resonance (eg, proton) imaging, orbit, face, and/or neck; without contrast material(s)], Drainage of abdomen [Peritoneocentesis, abdominal paracentesis, or peritoneal lavage (diagnostic or therapeutic); initial], Drainage of abdomen [Abdominal paracentesis (diagnostic or therapeutic); with imaging guidance], MRI of face [Magnetic resonance (eg, proton) imaging, orbit, face, and/or neck; without contrast material(s), followed by contrast material(s) and further sequences], Incision of neck [Bypass graft, with other than vein, transcervical retropharyngeal carotid-carotid, performed in conjunction with endovascular repair of descending thoracic aorta, by neck incision], Incision of neck [Thrombectomy, direct or with catheter; subclavian vein, by neck incision], Incision of neck [Embolectomy or thrombectomy, with or without catheter; carotid, subclavian or innominate artery, by neck incision], Computed tomography, abdomen [Computed tomography, abdomen], CT of abdominal vascular structures [Computed tomographic angiography, abdomen, with contrast material(s), including noncontrast images, if performed, and image postprocessing], CT of abdomen with contrast [Computed tomography, abdomen; with contrast material(s)], Head and neck incision [Repair, congenital arteriovenous fistula; head and neck], CT of upper limb [Computed tomographic angiography, upper extremity, with contrast material(s), including noncontrast images, if performed, and image postprocessing], CT of upper limb [Computed tomography, upper extremity; without contrast material, followed by contrast material(s) and further sections]]","[0.1268, 0.1351, 0.1351, 0.1526, 0.1669, 0.1869, 0.1899, 0.1899, 0.1882, 0.1952, 0.1952, 0.1952, 0.1955, 0.1962, 0.1986, 0.2006, 0.2007, 0.2007]"


In [ ]:
text = 'Left heart cath'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 15.5 ms, sys: 1.43 ms, total: 16.9 ms
Wall time: 214 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Left heart cath,0,14,93462,"[93462, 93607, 0281T, 93514, 93452, 78428, 93566, 93565, 93568, 93564, 93567]","[Catheterisation of left heart [Left heart catheterization by transseptal puncture through intact septum or by transapical puncture (List separately in addition to code for primary procedure)], Left ventricular recording (Deprecated) [Left ventricular recording (Deprecated)], Plication of left atrial appendage [Percutaneous transcatheter closure of the left atrial appendage with implant, including fluoroscopy, transseptal puncture, catheter placement(s), left atrial angiography, left atrial appendage angiography, radiological supervision and interpretation], Left heart catheterization by left ventricular puncture [Left heart catheterization by left ventricular puncture], Ventriculography of left ventricle [Left heart catheterization including intraprocedural injection(s) for left ventriculography, imaging supervision and interpretation, when performed], Cardiac shunt detection (procedure) [Cardiac shunt detection], Cardiac catheterisation [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for selective right ventricular or right atrial angiography (List separately in addition to code for primary procedure)], Cardiac catheterisation [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for selective left ventricular or left atrial angiography (List separately in addition to code for primary procedure)], Cardiac catheterisation [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for pulmonary angiography (List separately in addition to code for primary procedure)], Cardiac catheterisation [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for selective opacification of aortocoronary venous or arterial bypass graft(s) (eg, aortocoronary saphenous vein, free radial artery, or free ma], Cardiac catheterisation [Injection procedure during cardiac catheterization including imaging supervision, interpretation, and report; for supravalvular aortography (List separately in addition to code for primary procedure)]]","[0.1218, 0.1272, 0.1678, 0.1690, 0.1799, 0.1914, 0.1874, 0.1874, 0.1874, 0.1874, 0.1874]"


In [ ]:
text = 'bladder cancer'

%time get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)

CPU times: user 17.3 ms, sys: 176 µs, total: 17.5 ms
Wall time: 3 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,bladder cancer,0,13,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 139850000, 162582000, 315269000, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, suspected bladder cancer, suspected bladder cancer, suspected bladder cancer, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 0.0539, 0.0666, 0.0666, 0.0666, 0.0809, 0.0881, 0.0904, 0.0904, 0.0904, 0.0880, 0.0880, 0.0913, 0.0978, 0.1080]","[Malignant tumor of urinary bladder, NGB - New growth of bladder, Carcinoma of urinary bladder, invasive, Carcinoma of bladder, Bladder Ca, Carcinoma bladder, Carcinoma of urinary bladder, superficial, Adenocarcinoma of bladder, Suspected bladder cancer, Suspected bladder cancer, Suspected bladder cancer, Transitional cell carcinoma of bladder, Transitional cell carcinoma of bladder, Squamous cell carcinoma of bladder, Cancer in situ of urinary bladder, Tumor of bladder neck]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Observation, Condition, Condition, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Context-dependent, Context-dependent, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]"


In [ ]:
text = 'schizophrenia'

%time get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)

CPU times: user 16.1 ms, sys: 1.89 ms, total: 18 ms
Wall time: 3.01 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,schizophrenia,0,12,58214004,"[58214004, 83746006, 191567000, 191529003, 16990005, 35252006, 137730004, 160328009, 64905009, 161468000, 138739000, 191526005, 12939007, 76834001, 154868009, 76839006, 191542003]","[schizophrenia, chronic schizophrenia, schizoaffective schizophrenia, subchronic schizophrenia, subchronic schizophrenia, disorganized schizophrenia, fh: schizophrenia, fh: schizophrenia, paraphrenic schizophrenia, h/o: schizophrenia, h/o: schizophrenia, schizophrenic disorders, chronic disorganised schizophrenia, retired procedure, catatonic schizophrenia, catatonic schizophrenia, catatonic schizophrenia]","[0.0000, 0.0387, 0.0481, 0.0498, 0.0498, 0.0532, 0.0597, 0.0597, 0.0663, 0.0653, 0.0653, 0.0662, 0.0683, 0.0694, 0.0694, 0.0694, 0.0694]","[Schizophrenia, Chronic schizophrenia, Schizoaffective schizophrenia, Subchronic schizophrenia, Subchronic schizophrenia, Disorganized schizophrenia, FH: Schizophrenia, FH: Schizophrenia, Paranoid schizophrenia, H/O: schizophrenia, H/O: schizophrenia, Schizophrenic disorders, Chronic disorganized schizophrenia, Retired procedure, Catatonic schizophrenia, Catatonic schizophrenia, Catatonic schizophrenia]","[Condition, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Condition, Observation, Observation, Condition, Condition, Procedure, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Context-dependent, Clinical Finding, Context-dependent, Context-dependent, Clinical Finding, Clinical Finding, Procedure, Clinical Finding, Clinical Finding, Clinical Finding]"


In [ ]:
text = 'FLT3 gene mutation analysis'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 50.4 ms, sys: 15.1 ms, total: 65.5 ms
Wall time: 6.89 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,FLT3 gene mutation analysis,0,26,LP228450-5,"[LP228450-5, LP310331-6, LP228432-3, LP228446-3, LP228445-5, LP229390-2, LP228738-3, LP229796-0, LP229155-9, LP228431-5, LP229833-1, LP228570-0, LP229609-5, LP227517-2, 92843-2, LP229667-3, LP228546-0, LP228828-2, LP229608-7, LP228488-5, LP228434-9, LP228430-7, LP228825-8, LP228479-4]","[flt3 gene targeted mutation analysis, flt3 gene p.asp835 mutations, fgfr3 gene targeted mutation analysis, flna gene targeted mutation analysis, flcn gene targeted mutation analysis, pou3f4 gene targeted mutation analysis, itgb3 gene targeted mutation analysis, tgfb3 gene targeted mutation analysis, opa3 gene targeted mutation analysis, fgfr3 gene mutations tested for, traf3 gene targeted mutation analysis, gpc3 gene targeted mutation analysis, sh3tc2 gene targeted mutation analysis, abca3 gene targeted mutation analysis, flt3 p.d835 mutations, spg3a gene targeted mutation analysis, gjb3 gene targeted mutation analysis, lamb3 gene targeted mutation analysis, sh3bp2 gene targeted mutation analysis, fxn gene targeted mutation analysis, fh gene targeted mutation analysis, fgfr2 gene+fgfr3 gene targeted mutation analysis, lama3 gene targeted mutation analysis, fshd gene targeted mutation analysis]","[0.0354, 0.1032, 0.1021, 0.1111, 0.1135, 0.1258, 0.1249, 0.1285, 0.1255, 0.1316, 0.1287, 0.1328, 0.1346, 0.1315, 0.1358, 0.1351, 0.1352, 0.1339, 0.1390, 0.1347, 0.1318, 0.1387, 0.1362, 0.1360]"


In [ ]:
text = 'Hematocrit'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 13.8 ms, sys: 5.01 ms, total: 18.8 ms
Wall time: 1.23 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Hematocrit,0,9,LP15101-6,"[LP15101-6, LP308151-2, 32354-3, 20570-8, 11153-4, LP74090-9, 13508-7, %{of'HGB}, 42908-4, LP29006-1, 11559-2, 71832-0, LP16437-3, LP14449-0, 41654-5, LP35956-9, 41986-1, LP16428-2, 48703-3, 71828-8, 55103-6, 8478-0, 41655-2, LP30931-7]","[hematocrit, hematocrit/hemoglobin, hematocrit [volume fraction] of arterial blood, hematocrit [volume fraction] of blood, hematocrit [volume fraction] of body fluid, hematocrit test status, microhematocrit, percent hemoglobin, hematocrit [volume fraction] of capillary blood, blood volume, fractional hemoglobin, hematocrit [pure volume fraction] of arterial blood, hemoglobin i, hemoglobin, hematocrit [volume fraction] of venous blood, hematocrit test status/results, hematocrit test status/results, hemoglobin f, hematocrit [volume fraction] of blood by estimated, hematocrit [pure volume fraction] of body fluid, blood output, mean blood pressure, hematocrit [volume fraction] of mixed venous blood, hemoglobin m]","[0.0000, 0.0506, 0.0590, 0.0625, 0.0675, 0.0740, 0.0737, 0.0996, 0.1035, 0.1010, 0.1060, 0.1075, 0.1082, 0.1122, 0.1139, 0.1162, 0.1162, 0.1166, 0.1196, 0.1194, 0.1167, 0.1193, 0.1245, 0.1245]"


In [ ]:
text = 'urine test'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 8.7 ms, sys: 9.88 ms, total: 18.6 ms
Wall time: 194 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,urine test,0,9,80334-6,"[80334-6, 18729-4, 9189-2, 9191-8, 9192-6, 9190-0, 9193-4, 9187-6, 80523-4, 9188-4, 50564-4, 46903-1, 46924-7, 46942-9, 9195-9, 11279-7, 9197-5, 19159-3, 80332-0, 46944-5, 46926-2, 46905-6, 9044-9, 80522-6, 81323-8]","[Urine collection device, Urinalysis studies, Fluid output.urine, Fluid output.urine, Fluid output.urine, Fluid output.urine, Fluid output.urine, Fluid output.urine, Fluid output.urine, Fluid output.urine, Urinalysis panel, Urinary function.behavior, Urinary function.behavior, Urinary function.behavior, Fluid output.urine in and out urethral catheter, Urine sediment comments, Fluid output.urine^post void, Urinalysis specimen collection method, Voiding pattern, Urinary function.status, Urinary function.status, Urinary function.status, Urinary bladder irrigation tube site, Urine flow.mean, Urine collection associated observations panel]","[0.0797, 0.0839, 0.1115, 0.1115, 0.1115, 0.1115, 0.1115, 0.1115, 0.1115, 0.1115, 0.1183, 0.1221, 0.1221, 0.1221, 0.1328, 0.1369, 0.1376, 0.1380, 0.1379, 0.1367, 0.1367, 0.1367, 0.1412, 0.1454, 0.1552]"


In [14]:
text = ["Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type"]

%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 11.8 ms, sys: 1.87 ms, total: 13.6 ms
Wall time: 299 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,"Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type",0,104,L8001,"[L8001, L8002, L8000, L8033, L8032, L8020, S8080, G9071, L8010, G9131, S2068, S2067, S2066, G9073, G9072, G9075, K1005, G9074, L8031, L8015, E0602, L8600, E0604, G9114, L8030]","[Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type, Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, bilateral, any size, any type, Breast prosthesis, mastectomy bra, without integrated breast prosthesis form, any size, any type, Nipple prosthesis, custom fabricated, reusable, any material, any type, each, Nipple prosthesis, prefabricated, reusable, any type, each, Breast prosthesis, mastectomy form, Scintimammography (radioimmunoscintigraphy of the breast), unilateral, including supply of radiopharmaceutical, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage i or stage iia-iib; or t3, n1, m0; and er and/or pr positive; with no evidence of disease progression, r..., Breast prosthesis, mastectomy sleeve, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; extent of disease unknown, staging in progress, or not listed (for use in a medicare-approved demonstration pr..., Breast reconstruction with deep inferior epigastric perforator (diep) flap or superficial inferior epigastric artery (siea) flap, including harvesting of the flap, microvascular transfer, closure of donor site and shaping the flap into a breast, unilat..., Breast reconstruction of a single breast with ""stacked"" deep inferior epigastric perforator (diep) flap(s) and/or gluteal artery perforator (gap) flap(s), including harvesting of the flap(s), microvascular transfer, closure of donor site(s) and shaping..., Breast reconstruction with gluteal artery perforator (gap) flap, including harvesting of the flap, microvascular transfer, closure of donor site and shaping the flap into a breast, unilateral, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage iiia-iiib; and not t3, n1, m0; and er and/or pr positive; with no evidence of disease progression, recur..., Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage i, or stage iia-iib; or t3, n1, m0; and er and pr negative; with no evidence of disease progression, rec..., Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; m1 at diagnosis, metastatic, locally recurrent, or progressive (for use in a medicare-approved demonstration p..., Disposable collection and storage bag for breast milk, any size, any type, each, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage iiia-iiib; and not t3, n1, m0; and er and pr negative; with no evidence of disease progression, recurren..., Breast prosthesis, silicone or equal, with integral adhesive, External breast prosthesis garment, with mastectomy form, post mastectomy, Breast pump, manual, any type, Implantable breast prosthesis, silicone or equal, Breast pump, hospital grade, electric (ac and / or dc), any type, Oncology; disease status; ovarian cancer, limited to epithelial cancer; pathologic stage ia-b (grade 2-3); or stage ic (all grades); or stage ii; without evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonst..., Breast prosthesis, silicone or equal, without integral adhesive]","[0.0000, 0.1269, 0.1572, 0.1733, 0.1818, 0.1990, 0.2050, 0.2146,

In [19]:
text= ["Dietary consultation for carbohydrate counting for type I diabetes."]
%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 11.3 ms, sys: 2.24 ms, total: 13.6 ms
Wall time: 238 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dietary consultation for carbohydrate counting for type I diabetes.,0,66,S9465,"[S9465, S9434, S9460, E0787, G0246, G8777, S9435, G0245, B4157, B5200, B4162, G8725, G8485, B4155, B4149, G0343, A4236, S9470, B5100, A4234, G9147, A4235, B5000, G0342, S9455]","[Diabetic management program, dietitian visit, Modified solid food supplements for inborn errors of metabolism, Diabetic management program, nurse visit, External ambulatory infusion pump, insulin, dosage rate adjustment using therapeutic continuous glucose sensing, Follow-up physician evaluation and management of a diabetic patient with diabetic sensory neuropathy resulting in a loss of protective sensation (lops) to include at least the following: (1) a patient history, (2) a physical examination that includes:..., Diabetes screening test performed, Medical foods for inborn errors of metabolism, Initial physician evaluation and management of a diabetic patient with diabetic sensory neuropathy resulting in a loss of protective sensation (lops) which must include: (1) the diagnosis of lops, (2) a patient history, (3) a physical examination that..., Enteral formula, nutritionally complete, for special metabolic needs for inherited disease of metabolism, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, stress-branch chain amino acids-freamine-hbc-premix, Enteral formula, for pediatrics, special metabolic needs for inherited disease of metabolism, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Fasting lipid profile performed (triglycerides, ldl-c, hdl-c and total cholesterol), I intend to report the diabetes mellitus (dm) measures group, Enteral formula, nutritionally incomplete/modular nutrients, includes specific nutrients, carbohydrates (e.g., glucose polymers), proteins/amino acids (e.g., glutamine, arginine), fat (e.g., medium chain triglycerides) or combination, administered thro..., Enteral formula, manufactured blenderized natural foods with intact nutrients, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Laparotomy for islet cell transplant, includes portal vein catheterization and infusion, Replacement battery, silver oxide, for use with medically necessary home blood glucose monitor owned by patient, each, Nutritional counseling, dietitian visit, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, hepatic, hepatamine-premix, Replacement battery, alkaline, j cell, for use with medically necessary home blood glucose monitor owned by patient, each, Outpatient intravenous insulin treatment (oivit) either pulsatile or continuous, by any means, guided by the results of measurements for: respiratory quotient; and/or, urine urea nitrogen (uun); and/or, arterial, venous or capillary glucose; and/or pot..., Replacement battery, lithium, for use with medically necessary home blood glucose monitor owned by patient, each, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, renal-aminosyn-rf, nephramine, renamine-premix, Laparoscopy for islet cell transplant, includes portal vein catheterization and infusion, Diabetic management program, group session]","[0.2426, 0.3054, 0.3147, 0.3332, 0.3361, 0.3201, 0.3398, 0.3528, 0.3355, 0.3480, 0.3399, 0.3615, 0.3432, 0.3561, 0.3535, 0.3669, 0.3641, 0.3510, 0.3661, 0.3631, 0.3738, 0.3677, 0.3712, 0.3795, 0.3816]"


In [23]:
text= ["Psychiatric consultation for alcohol withdrawal and dependance."]
%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 13.8 ms, sys: 3.89 ms, total: 17.7 ms
Wall time: 218 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Psychiatric consultation for alcohol withdrawal and dependance.,0,62,H0010,"[H0010, H0011, H0007, H0012, H0013, H0008, T1006, G9621, H0009, S9485, H2036, H0005, G8959, H2034, H0014, H0006, H2018, T1007, H0022, H2031, G2121, T1012, G9584, H2013, H0021]","[Alcohol and/or drug services; sub-acute detoxification (residential addiction program inpatient), Alcohol and/or drug services; acute detoxification (residential addiction program inpatient), Alcohol and/or drug services; crisis intervention (outpatient), Alcohol and/or drug services; sub-acute detoxification (residential addiction program outpatient), Alcohol and/or drug services; acute detoxification (residential addiction program outpatient), Alcohol and/or drug services; sub-acute detoxification (hospital inpatient), Alcohol and/or substance abuse services, family/couple counseling, Patient identified as an unhealthy alcohol user when screened for unhealthy alcohol use using a systematic screening method and received brief counseling, Alcohol and/or drug services; acute detoxification (hospital inpatient), Crisis intervention mental health services, per diem, Alcohol and/or other drug treatment program, per diem, Alcohol and/or drug services; group counseling by a clinician, Clinician treating major depressive disorder communicates to clinician treating comorbid condition, Alcohol and/or drug abuse halfway house services, per diem, Alcohol and/or drug services; ambulatory detoxification, Alcohol and/or drug services; case management, Psychosocial rehabilitation services, per diem, Alcohol and/or substance abuse services, treatment plan development and/or modification, Alcohol and/or drug intervention service (planned facilitation), Mental health clubhouse services, per diem, Psychosis, depression, anxiety, apathy, and impulse control disorder assessed, Alcohol and/or substance abuse services, skills development, Patient evaluated for risk of misuse of opiates by using a brief validated instrument (e.g., opioid risk tool, soapp-r) or patient interviewed at least once during opioid therapy, Psychiatric health facility service, per diem, Alcohol and/or drug training service (for staff and personnel not employed by providers)]","[0.1938, 0.2086, 0.2104, 0.2134, 0.2177, 0.2423, 0.2430, 0.2509, 0.2554, 0.2716, 0.2658, 0.2695, 0.2716, 0.2755, 0.2933, 0.2799, 0.3020, 0.2915, 0.2985, 0.3063, 0.3060, 0.3053, 0.3191, 0.3205, 0.3181]"


In [ ]:
# medical device
text = 'X-Ray'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 8.44 ms, sys: 6.05 ms, total: 14.5 ms
Wall time: 470 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,X-Ray,0,4,C0043303,"[C0043303, C0182605, C0184486, C0183264, C0221564, C0183263, C3880096, C3878814, C3874200, C0000766, C0750687, C3880545, C3873785]","[x-ray film, x-ray machine, x-ray tubes, x-ray aprons, specific x-ray findings, x-ray shield, device, x-ray system tube (physical object), x-ray image intensifier (physical object), cephalometric x-ray system (physical object), x-ray abnormal, cabinet,x-ray system, x-ray system cover (physical object), specimen x-ray system (physical object)]","[0.0637, 0.1052, 0.1160, 0.1400, 0.1415, 0.1425, 0.1423, 0.1463, 0.1529, 0.1442, 0.1503, 0.1518, 0.1563]"


In [ ]:
# Injuries & poisoning
text = 'out-of-date food poisoning'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 539 µs, sys: 17.6 ms, total: 18.1 ms
Wall time: 488 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,out-of-date food poisoning,0,25,C0016479,"[C0016479, C0349783, C0411266, C0161721, C0417672, C0567797, C0411267, C1281398, C0178496, C4743928, C0343360, C3665356, C4724202, C1820738, C0565937, C4743931]","[food poisoning (disorder), infective food poisoning (disorder), chemical food poisoning (disorder), toxic effect of noxious substance eaten as food (disorder), accident due to lack of food, pectin poisoning of undetermined intent (disorder), heavy metal food poisoning (disorder), inanition due to lack of food (disorder), bacterial food poisoning (disorder), grade 2 feeding intolerance, viral food poisoning (disorder), history of food intolerance, feeding intolerance, feeding intolerance, food burn of skin (disorder), grade 5 feeding intolerance, adverse event]","[0.1007, 0.1104, 0.1117, 0.1331, 0.1431, 0.1496, 0.1519, 0.1546, 0.1548, 0.1585, 0.1625, 0.1618, 0.1624, 0.1624, 0.1645, 0.1660]"


In [ ]:
# clinical findings
text = 'type two diabetes mellitus'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 9.09 ms, sys: 4.58 ms, total: 13.7 ms
Wall time: 481 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,type two diabetes mellitus,0,25,C4014362,"[C4014362, C3532488, C1320657, C1313937, C0455488, C2733146, C4017629, C3532489, C4538688, C3275844, C0241863, C3280267, C1261139, C0421248, C5195213, C1317301, C3278636, C2675471, C3280359]","[type 2 diabetes mellitus (t2d), history of diabetes mellitus type 2 (situation), diabete type, fh: diabetes mellitus, pre-existing diabetes mellitus, type 2 diabetes mellitus uncontrolled, diabetes mellitus, type ii, digenic, history of diabetes mellitus type i, diabetes mellitus, type 2 (in some heterozygous adults), diabetes mellitus, type ii, susceptibility to, diabetic, type 2 diabetes (in some), diabetic relative, insulin diabetic, increased risk of type 2 diabetes, diabetes status, neonatal insulin-dependent diabetes mellitus, microvascular complications of diabetes, susceptibility to, 2, diabetes mellitus (in some patients)]","[0.0438, 0.0568, 0.0618, 0.0697, 0.0780, 0.0891, 0.0910, 0.0883, 0.0937, 0.0940, 0.0939, 0.0971, 0.1031, 0.1062, 0.1087, 0.1104, 0.1116, 0.1201, 0.1147]"


In [ ]:
text = 'chest pain '

%time get_codes (mesh_lp, text, vocab='mesh_code')

CPU times: user 72.4 ms, sys: 9.09 ms, total: 81.5 ms
Wall time: 8.97 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,chest pain,0,10,D002637,"[D002637, D059350, D019547, D020069, D015746, D000072716, D005157, D059265, D001416, D048949, D021501, D058447, D010146, D059390, C563475, D059352, D053591, D059787, D009209, D010148, D058748, D020918, D017699, D059388, D005409]","[Chest Pain, Chronic Pain, Neck Pain, Shoulder Pain, Abdominal Pain, Cancer Pain, Facial Pain, Visceral Pain, Back Pain, Labor Pain, Flank Pain, Eye Pain, Pain, Breakthrough Pain, Paroxysmal Extreme Pain Disorder, Musculoskeletal Pain, Pain, Referred, Acute Pain, Myofascial Pain Syndromes, Pain, Intractable, Pain Perception, Complex Regional Pain Syndromes, Pelvic Pain, Pelvic Girdle Pain, Flail Chest]","[0.0000, 0.0577, 0.0587, 0.0601, 0.0658, 0.0704, 0.0712, 0.0741, 0.0766, 0.0778, 0.0794, 0.0827, 0.0845, 0.0914, 0.0941, 0.0935, 0.0957, 0.1023, 0.1078, 0.1060, 0.1109, 0.1166, 0.1187, 0.1189, 0.1199]"


In [7]:
text = 'pericardectomy'

%time get_codes (mesh_lp, text, vocab='mesh_code')

CPU times: user 67.9 ms, sys: 5.98 ms, total: 73.8 ms
Wall time: 8.17 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,pericardectomy,0,13,D010492,"[D010492, D011670, D018700, D014068, D000541, D005927, D020884, D064727, D010611, D011672, D059526, D001062, D002431, D011013, D013156, D032801, D001047, D020519, D000074885, D038421, D010180, D007825, D000074406, D017128, D056346]","[Pericardiectomy, Pulpectomy, Pleurodesis, Tonsillectomy, Alveolectomy, Glossectomy, Colpotomy, Posterior Capsulotomy, Pharyngectomy, Pulpotomy, Foraminotomy, Appendectomy, Cecostomy, Pneumonectomy, Splenectomy, Iridectomy, Apicoectomy, Pericardiocentesis, Amniotomy, Hemispherectomy, Pancreatectomy, Laryngectomy, Mastoidectomy, Embolectomy, Sternotomy]","[0.0732, 0.1287, 0.1499, 0.1540, 0.1573, 0.1582, 0.1645, 0.1622, 0.1660, 0.1676, 0.1729, 0.1697, 0.1744, 0.1774, 0.1753, 0.1757, 0.1783, 0.1812, 0.1803, 0.1726, 0.1799, 0.1813, 0.1852, 0.1909, 0.1867]"


In [ ]:
text = 'bladder cancer'

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 4.88 ms, sys: 8.95 ms, total: 13.8 ms
Wall time: 132 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,HP:0009725,"[HP:0009725, HP:0002862, HP:0006740, HP:0030409, HP:0100517, HP:0000015, HP:0100516, HP:0009726, HP:0004375, HP:0004321, HP:0032465, HP:0010786, HP:0010474, HP:0010784, HP:0006762, HP:0005584, HP:0025489, HP:0010477]","[bladder cancer, bladder carcinoma, transitional cell carcinoma of the bladder, renal urothelial carcinoma, neoplasia of the urethra, bladder diverticulum, ureter, cancer of, kidney cancer, nervous system cancer, bladder fistula, bladder trabeculation, urinary tract neoplasia, bladder stones, uterine cancer, renal pelvic carcinoma, renal carcinoma, bladder duplication, aplasia of the bladder]","[0.0000, 0.0503, 0.0905, 0.1612, 0.1729, 0.1738, 0.1839, 0.1886, 0.1924, 0.1892, 0.1930, 0.1903, 0.1904, 0.1938, 0.1987, 0.2013, 0.2006, 0.2039]"


In [ ]:
text = 'bipolar disorder'

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 10.9 ms, sys: 7.45 ms, total: 18.3 ms
Wall time: 127 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bipolar disorder,0,15,HP:0007302,"[HP:0007302, HP:0100753, HP:0012076, HP:0000708, HP:0033516, HP:0012077, HP:0000709, HP:0012075, HP:0000716, HP:0000722, HP:0002134, HP:0011511, HP:0000725, HP:0025102, HP:0002072, HP:0011223, HP:0033512, HP:0100022, HP:0001345, HP:0002269]","[bipolar disorder, schizophrenia, borderline personality disorder, psychiatric disorders, benzodiazepine dependence, histrionic personality disorder, psychosis, personality disorder, depressive disorder, obsessive compulsive disorder, basal ganglia disease, macular schisis, psychotic episodes, basal ganglia dysgenesis, choreatic disease, metopic depression, stimulant dependence, movement disorder, psychotic mentation, migrational brain disorder]","[0.0000, 0.1488, 0.1486, 0.1577, 0.1740, 0.1731, 0.1784, 0.1764, 0.1786, 0.1830, 0.1925, 0.2145, 0.2192, 0.2182, 0.2376, 0.2325, 0.2325, 0.2330, 0.2443, 0.2393]"


In [ ]:
text = 'schizophrenia '

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 6.34 ms, sys: 7.83 ms, total: 14.2 ms
Wall time: 124 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,schizophrenia,0,13,HP:0100753,"[HP:0100753, HP:0000709, HP:0007302, HP:0000708, HP:0000725, HP:0011511, HP:0012077, HP:0001345, HP:0030325, HP:0012075, HP:0000722, HP:0033512, HP:0012076, HP:0100923, HP:0002072, HP:0033715, HP:0010891, HP:0011626, HP:0100656, HP:0033516, HP:0008404, HP:0002269]","[schizophrenia, psychosis, bipolar disorder, psychiatric disorders, psychotic episodes, macular schisis, histrionic personality disorder, psychotic mentation, cervicomedullary schisis, personality disorder, obsessive compulsive disorder, stimulant dependence, borderline personality disorder, clavicular sclerosis, choreatic disease, mesial temporal sclerosis, scheuermann disease, scimitar syndrome, thoracoabdominal schisis, benzodiazepine dependence, onychodystrophy, migrational brain disorder]","[0.0000, 0.1172, 0.1488, 0.1599, 0.1877, 0.1898, 0.1923, 0.2033, 0.2163, 0.2113, 0.2166, 0.2257, 0.2202, 0.2345, 0.2431, 0.2389, 0.2497, 0.2438, 0.2578, 0.2483, 0.2604, 0.2455]"


In [ ]:
icd_chunks = ['advanced liver disease',
'advanced lung disease',
'basal cell carcinoma of skin',
'acute maxillary sinusitis',
'chronic kidney disease stage',
'diabetes mellitus type 2',
'lymph nodes of multiple sites',
'other chronic pain',
'severe abdominal pain',
'squamous cell carcinoma of skin',
'type 2 diabetes mellitus']

snomed_chunks= ['down syndrome', 'adenocarcinoma', 'aortic valve stenosis',
       'atherosclerosis', 'atrial fibrillation',
       'hypertension', 'lung cancer', 'seizure',
       'squamous cell carcinoma', 'stage IIIB', 'mediastinal lymph nodes']

In [ ]:
from IPython.display import display

for chunk in icd_chunks:

    print ('>> ',chunk)
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))

>>  advanced liver disease


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,advanced liver disease,0,21,K7460,"[K7460, K769, K721, K746, R945, Z871, K7402, K729, K7290, K7689, D489, D499, R748, K719]","[advanced cirrhosis, chronic liver disease, chronic hepatic failure, cirrhosis and chronic liver disease, abnormal liver function, h/o: liver disease, hepatic fibrosis, advanced fibrosis, liver function failure, liver failure, abnormal hepatic function, tumor of advanced extent, tumour of advanced extent, elevated liver enzymes level, toxic liver disease]","[0.0704, 0.1057, 0.1081, 0.1263, 0.1299, 0.1324, 0.1338, 0.1319, 0.1334, 0.1425, 0.1468, 0.1477, 0.1487, 0.1517]"


>>  advanced lung disease


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,advanced lung disease,0,20,D499,"[D499, D489, J449, H359, J849, J628, K7460, J399, I5083, J961, J989, I509, J8410, R54, J984, I279]","[tumour of advanced extent, tumor of advanced extent, severe chronic obstructive pulmonary disease, advanced retinal disease, chronic infiltrative lung disease, chronic lung disease, advanced cirrhosis, upper respiratory disease, high output heart failure, chronic respiratory failure, chronic respiratory system disease, cardiac failure, progressive lung fibrosis, advanced age, chronic lung disease (disorder), chronic pulmonary heart disease]","[0.1284, 0.1333, 0.1329, 0.1353, 0.1355, 0.1373, 0.1455, 0.1471, 0.1468, 0.1537, 0.1565, 0.1572, 0.1588, 0.1597, 0.1641, 0.1644]"


>>  basal cell carcinoma of skin


...

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,severe abdominal pain,0,20,R1084,"[R1084, R101, R109, K388, R104, R1033, R102, F454, K805, R529, R52, M2551]","[generalized abdominal pain, upper abdominal pain, intractable abdominal pain, colicky abdominal pain, generalized colicky abdominal pain, central abdominal pain, acute abdominal pain, psychosomatic abdominal pain, recurrent abdominal pain, severe pain, heavy pain, shoulder pain from abdomen]","[0.0196, 0.0208, 0.0208, 0.0318, 0.0322, 0.0352, 0.0370, 0.0370, 0.0408, 0.0424, 0.0489, 0.0507]"


>>  squamous cell carcinoma of skin


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma of skin,0,30,C449,"[C449, C443, D049, C4492, C4452, C4499, C4432, C809, C4442, C4A70, C444, C4491, C799]","[squamous cell carcinoma of skin, squamous cell carcinoma of skin of face, squamous cell carcinoma in situ of skin, scc - squamous cell carcinoma of skin, squamous cell carcinoma of skin of trunk, mucoepidermoid carcinoma of skin, squamous cell carcinoma of skin of and unsp parts of face, squamous cell carcinoma, squamous cell carcinoma of skin of neck, trabecular cell carcinoma of skin, squamous cell carcinoma of scalp, basal cell carcinoma of skin, squamous cell carcinomatosis]","[0.0000, 0.0244, 0.0289, 0.0333, 0.0337, 0.0420, 0.0519, 0.0547, 0.0564, 0.0600, 0.0623, 0.0620, 0.0638]"


>>  type 2 diabetes mellitus


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus,0,23,E119,"[E119, E118, E113, O2411, E139, E117, E1143, E109, E1169, E114, E112, E1151, E1162]","[type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, secondary diabetes mellitus, multiple complications of type 2 diabetes mellitus, neurological disorder with type 2 diabetes mellitus, diabetes mellitus, diabetic dyslipidemia associated with type 2 diabetes mellitus, neurological disorder with diabetes type 2, kidney disorder associated with type 2 diabetes mellitus, peripheral circulatory disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus with skin complications]","[0.0000, 0.0212, 0.0357, 0.0360, 0.0397, 0.0410, 0.0455, 0.0462, 0.0499, 0.0523, 0.0537, 0.0548, 0.0549]"


In [ ]:
for chunk in snomed_chunks:

    print ('>> ',chunk)
    display(get_codes (snomed_lp, chunk, vocab='snomed_code', aux_label=True))

>>  down syndrome


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,down syndrome,0,12,41040004,"[41040004, 157019002, 79385002, 408338009, 247545005, 205618003, 254263008, 95429007, 711461002, 25088000, 45056002, 155375008, 266308000, 20741006, 29696001, 711466007, 230675008, 22994000, 255518004, 248199009, 206891002]","[down syndrome, down's syndrome, lowe syndrome, suspected downs syndrome, downy hair, down's syndrome nos, down's syndrome nos, shortened trunk (disorder), decreased lower face height, dimpled structure, abnormal shortened structure, dropsy, dropsy, dropsy, downward displacement, decreased lower anterior face height, slow channel syndrome (disorder), decreased reflex, downward, down facies, [d]dropsy]","[0.0000, 0.0341, 0.0753, 0.0790, 0.0966, 0.1007, 0.1007, 0.0995, 0.1016, 0.1054, 0.1059, 0.1066, 0.1066, 0.1066, 0.1068, 0.1114, 0.1116, 0.1132, 0.1114, 0.1132, 0.1127]","[Complete trisomy 21 syndrome, Down's syndrome, Lowe syndrome, Suspected Downs syndrome, Downy hair, Down's syndrome NOS, Trisomy 21 NOS, Shortened trunk, Decreased lower face height, Dimple, Abnormal shortening, Right heart failure, Dropsy, Hydrops, Prolapse, Decreased lower anterior face height, Slow channel syndrome, Hyporeflexia, Down, Down's facies, [D]Dropsy]","[Condition, Condition, Condition, Observation, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Condition, Condition, Observation, Observation, Condition, Condition, Condition, Observation, Condition, Observation]","[Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Morph Abnormality, Morph Abnormality, Clinical Finding, Clinical Finding, Morph Abnormality, Morph Abnormality, Clinical Finding, Clinical Finding, Clinical Finding, Qualifier Value, Clinical Finding, Context-dependent]"


>>  adenocarcinoma


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,adenocarcinoma,0,13,443961001,"[443961001, 35917007, 403902008, 59367005, 72495009, 58069009, 450948005, 45410002, 423595004, 32968003, 189578007, 134297006, 11671000, 302827006, 86293007, 189614008, 28558000, 89740008]","[adenocarcinoma, adenocarcinoma, adenosquamous carcinoma, adenosquamous carcinoma, mucous adenocarcinoma, ceruminous adenocarcinoma, serrated adenocarcinoma, acinar adenocarcinoma, adenocarcinoma carcinomatosis, inflammatory adenocarcinoma, [m]adenocarcinomas, adenoma and adenocarcinoma, adenocystic carcinoma, adenocystic carcinoma, adenocarcinoid tumor, adenocarcinoid tumor, villous adenocarcinoma, lobular adenocarcinoma]","[0.0000, 0.0000, 0.0401, 0.0401, 0.0432, 0.0496, 0.0513, 0.0520, 0.0546, 0.0563, 0.0586, 0.0593, 0.0594, 0.0594, 0.0615, 0.0615, 0.0601, 0.0617]","[Malignant adenomatous neoplasm, Adenocarcinoma, Adenosquamous cell carcinoma, Adenosquamous carcinoma, Mucinous adenocarcinoma, Ceruminous adenocarcinoma, Serrated adenocarcinoma, Acinar cell carcinoma, Adenocarcinoma carcinomatosis, Inflammatory carcinoma, [M]Adenomas &/or adenocarcinomas, Adenoma and adenocarcinoma, Adenoid cystic carcinoma, Cylindroma, Adenocarcinoid tumor, Adenocarcinoid tumor, Villous adenocarcinoma, Lobular carcinoma]","[Condition, Observation, Condition, Observation, Observation, Observation, Observation, Observation, Condition, Observation, Condition, Observation, Observation, Observation, Observation, Condition, Observation, Observation]","[Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Qualifier Value, Morph Abnormality, Morph Abnormality, Morph Abnormality, Clinical Finding, Morph Abnormality, Morph Abnormality]"


>>  aortic valve stenosis


...

,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,squamous cell carcinoma,0,22,402815007,"[402815007, 28899001, 154605007, 400155002, 425303004, 37304002, 254651007, 89906000, 403904009, 189564006, 189565007, 59529006, 403905005, 403902008, 85956000, 400066006, 252989001]","[squamous cell carcinoma, squamous cell carcinoma, squamous cell carcinoma, squamous cell carcinoma - category, squamous cell carcinomatosis, basisquamous cell carcinoma, cutaneous squamous cell carcinoma, verrucous squamous cell carcinoma, verrucous squamous cell carcinoma, [m]papillomatosis nos, squamous cell carcinoma in situ, squamous cell carcinoma in situ, multiple squamous cell carcinomata, adenosquamous cell carcinoma, adenoid squamous cell carcinoma, intraepithelial squamous cell carcinoma, squamous carcinoma in situ]","[0.0000, 0.0000, 0.0000, 0.0176, 0.0191, 0.0348, 0.0398, 0.0455, 0.0455, 0.0468, 0.0468, 0.0468, 0.0495, 0.0498, 0.0504, 0.0566, 0.0594]","[Squamous cell carcinoma, Squamous cell carcinoma, Squamous cell carcinoma, Squamous cell carcinoma - category, Squamous cell carcinomatosis, Basosquamous carcinoma, Squamous cell carcinoma of skin, Verrucous carcinoma, Verrucous squamous cell carcinoma, [M]Papillomatosis NOS, Squamous cell carcinoma in situ, Squamous cell carcinoma in situ, Multiple squamous cell carcinomata, Adenosquamous cell carcinoma, Adenoid squamous cell carcinoma, Intraepithelial squamous cell carcinoma, Squamous carcinoma in situ]","[Condition, Observation, Condition, Observation, Condition, Observation, Condition, Observation, Condition, Observation, Condition, Observation, Condition, Condition, Observation, Condition, Observation]","[Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality]"


>>  stage IIIB


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,stage IIIB,0,9,261639007,"[261639007, 258226007, 261625007, 64062008, 307204003, 258376007, 73082003, 261638004, 258225006, 258224005, 261640009, 258227003, 45597001, 443815000, 50283003, 264674009, 307203009, 48105005]","[stage 3b, stage 3b, stage b3, clinical stage 3 b, grade 3b, group 3b, clinical stage 3 a, stage 3a, stage 3a, stage 3, stage 3c, stage 3c, group a3b, pt3a,b category, clinical stage iii, 3b, grade 3a, clinical stage 3 c]","[0.0644, 0.0644, 0.0715, 0.0771, 0.0933, 0.1085, 0.1143, 0.1185, 0.1185, 0.1239, 0.1290, 0.1290, 0.1326, 0.1356, 0.1327, 0.1339, 0.1339, 0.1361]","[Stage 3B, Stage 3b, Stage B3, Clinical stage 3 B, Grade 3b, Group 3B, Clinical stage 3 A, Stage 3A, Stage IIIa, Stage 3, Stage 3C, Stage IIIc, Blood group A>3<B, pT3a,b category, Clinical stage III, 3b, Grade 3a, Clinical stage 3 C]","[Observation, Observation, Observation, Condition, Observation, Observation, Condition, Observation, Observation, Observation, Observation, Observation, Condition, Condition, Condition, Observation, Observation, Condition]","[Qualifier Value, Qualifier Value, Qualifier Value, Clinical Finding, Qualifier Value, Qualifier Value, Clinical Finding, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Clinical Finding, Clinical Finding, Clinical Finding, Qualifier Value, Qualifier Value, Clinical Finding]"


>>  mediastinal lymph nodes


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,mediastinal lymph nodes,0,22,245276004,"[245276004, 181760002, 62683002, 62650009, 245275000, 178297005, 260926004, 281779003, 264099006, 364445008, 127931000, 301296002, 127248001, 734421006, 198237001, 20190004, 72372004, 281496003]","[mediastinal lymph nodes, mediastinal lymph node, mediastinal lymph node, stratum germinativum, subclavian lymph node group, mediastinal lymph nodes sampling, mediastinal node status, mediastinal lymphatic structure, mediastinal, mediastinal feature, retrotracheal lymph node (mediastinal), mediastinal observation, neoplasm of mediastinal lymph nodes, specimen from mass of mediastinal lymph node, mediastinal vein, mediastinal vein, pelvic peritoneum structure, structure of pelvic blood vessel]","[0.0000, 0.0226, 0.0226, 0.0342, 0.0458, 0.0511, 0.0635, 0.0670, 0.0782, 0.0777, 0.0813, 0.0849, 0.0879, 0.0892, 0.0882, 0.0882, 0.0875, 0.0961]","[Mediastinal lymph node group, Entire mediastinal lymph node, Mediastinal lymph node structure, Stratum germinativum, Subclavian lymph node group, Mediastinal lymph nodes sampling, Mediastinal node status, Structure of mediastinal lymphatic system, Mediastinal, Mediastinal feature, Retrotracheal lymph node, Mediastinal finding, Neoplasm of mediastinal lymph nodes, Specimen from mass of mediastinal lymph node, Entire mediastinal vein, Structure of mediastinal vein, Pelvic peritoneum structure, Structure of pelvic blood vessel]","[Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Procedure, Observation, Spec Anatomic Site, Observation, Observation, Spec Anatomic Site, Condition, Condition, Specimen, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site]","[Body Structure, Body Structure, Body Structure, Body Structure, Body Structure, Procedure, Attribute, Body Structure, Qualifier Value, Observable Entity, Body Structure, Clinical Finding, Clinical Finding, Specimen, Body Structure, Body Structure, Body Structure, Body Structure]"


In [ ]:
clinical_chunks = ['bladder cancer',
 'anemia in chronic kidney disease',
 'castleman disease',
 'congestive heart failure',
 'diabetes mellitus type 2',
 'lymph nodes of multiple sites',
 'malignant melanoma of skin',
 'malignant neoplasm of lower lobe, bronchus',
 'metastatic lung cancer',
 'secondary malignant neoplasm of bone',
 'type 2 diabetes mellitus',
 'type 2 diabetes mellitus/insulin',
 'unsp malignant neoplasm of lymph node']


for chunk in clinical_chunks:

    print ('>> ',chunk)
    
    print ('icd10cm_code')
    display(get_codes (hcc_lp, chunk, vocab='icd10cm_hcc_code', hcc=True))
    
    print ('snomed_code')
    display(get_codes (snomed_lp, chunk, vocab='snomed_code', aux_label=True))

>>  bladder cancer
icd10cm_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,bladder cancer,0,13,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 1, 1, 1, 1]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


snomed_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,bladder cancer,0,13,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 139850000, 162582000, 315269000, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, suspected bladder cancer, suspected bladder cancer, suspected bladder cancer, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 0.0539, 0.0666, 0.0666, 0.0666, 0.0809, 0.0881, 0.0904, 0.0904, 0.0904, 0.0880, 0.0880, 0.0913, 0.0978, 0.1080]","[Malignant tumor of urinary bladder, NGB - New growth of bladder, Carcinoma of urinary bladder, invasive, Carcinoma of bladder, Bladder Ca, Carcinoma bladder, Carcinoma of urinary bladder, superficial, Adenocarcinoma of bladder, Suspected bladder cancer, Suspected bladder cancer, Suspected bladder cancer, Transitional cell carcinoma of bladder, Transitional cell carcinoma of bladder, Squamous cell carcinoma of bladder, Cancer in situ of urinary bladder, Tumor of bladder neck]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Observation, Condition, Condition, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Context-dependent, Context-dependent, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]"


>>  anemia in chronic kidney disease
icd10cm_code


...

,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,type 2 diabetes mellitus/insulin,0,31,237599002,"[237599002, 366909003, 44054006, 368051000119109, 422014003, 422099009, 199230006, 137931000119102, 190384004, 421326000]","[insulin treated type 2 diabetes mellitus, insulin treated type 2 diabetes mellitus, type 2 diabetes mellitus, hyperglycemia due to type 2 diabetes mellitus (disorder), disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus (disorder), hyperlipidaemia due to type 2 diabetes mellitus, type ii diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus]","[0.0513, 0.0513, 0.0554, 0.0593, 0.0606, 0.0658, 0.0686, 0.0690, 0.0708, 0.0718]","[Insulin treated type 2 diabetes mellitus, Insulin treated Type 2 diabetes mellitus, Type 2 diabetes mellitus, Hyperglycemia due to type 2 diabetes mellitus, Disorder due to type 2 diabetes mellitus, Disorder of eye due to type 2 diabetes mellitus, Pre-existing type 2 diabetes mellitus, Hyperlipidemia due to type 2 diabetes mellitus, Type II diabetes mellitus, Disorder of nervous system due to type 2 diabetes mellitus]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]"


>>  unsp malignant neoplasm of lymph node
icd10cm_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,unsp malignant neoplasm of lymph node,0,36,C779,"[C779, C77, D360, C772, C775, D492, D219]","[secondary and unsp malignant neoplasm of lymph node, unsp, secondary and unspecified malignant neoplasm of lymph nodes, benign neoplasm of lymph node, secondary and unsp malignant neoplasm of intra-abd nodes, secondary and unsp malignant neoplasm of intrapelv nodes, neoplasm of lymph node, benign neoplasm of lymph vessels]","[0.0606, 0.0681, 0.0754, 0.0920, 0.1034, 0.0968, 0.1049]","[1, 0, 1, 1, 1, 1, 1]","[1, 0, 0, 1, 1, 0, 0]","[10, 0, 0, 8, 8, 0, 0]"


snomed_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,unsp malignant neoplasm of lymph node,0,36,188379006,"[188379006, 271526003, 188432007, 92197001, 189203003, 188706002, 92202008, 92156008, 188707006, 188713002, 92090002, 92203003, 188392009, 188393004, 178838007, 188709009, 188389005, 188715009, 173840000, 92199003, 92213006, 127232002]","[secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes nos, benign neoplasm of lymph node, benign neoplasm of lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of unspecified site, benign neoplasm of lymph nodes of neck, benign neoplasm of intrathoracic lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of head, face and neck, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrapelvic lymph nodes, benign neoplasm of epitrochlear lymph nodes, benign neoplasm of lymph nodes of upper limb, secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck nos, secondary and unspecified malignant neoplasm of intrathoracic lymph nodes, other specified operations on skull and/or spine and/or bone and/or joint, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrathoracic lymph nodes, secondary and unspecified malignant neoplasm of submental lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of multiple sites, other operation on stomach nos, benign neoplasm of lymph nodes of head (disorder), benign neoplasm of mediastinal lymph nodes, neoplasm of lymph node]","[0.0681, 0.0681, 0.0739, 0.0754, 0.0758, 0.0841, 0.0860, 0.0896, 0.0948, 0.0963, 0.0954, 0.0938, 0.0969, 0.0980, 0.0995, 0.0995, 0.0994, 0.1006, 0.1006, 0.0971, 0.0985, 0.0968]","[Secondary and unspecified malignant neoplasm of lymph nodes, Secondary and unspecified malignant neoplasm of lymph nodes, Secondary and unspecified malignant neoplasm of lymph nodes NOS, Benign neoplasm of lymph node, Benign neoplasm of lymph nodes, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of unspecified site, Benign neoplasm of lymph nodes of neck, Benign neoplasm of intrathoracic lymph nodes, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of head, face and neck, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrapelvic lymph nodes, Benign neoplasm of epitrochlear lymph nodes, Benign neoplasm of lymph nodes of upper limb, Secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck NOS, Secondary and unspecified malignant neoplasm of intrathoracic lymph nodes, Other specified operations on skull and/or spine and/or bone and/or joint, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrathoracic lymph nodes, Secondary and unspecified malignant neoplasm of submental lymph nodes, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of multiple sites, Other operation on stomach NOS, Benign neoplasm of lymph nodes of head, Benign neoplasm of mediastinal lymph nodes, Neoplasm of lymph node]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Procedure, Condition, Condition, Condition, Procedure, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Procedure, Clinical Finding, Clinical Finding, Clinical Finding, Procedure, Clinical Finding, Clinical Finding, Clinical Finding]"


##  How to integrate resolvers with NER models in the same pipeline


In [ ]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])

c2doc = Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")
  
sbert_resolver_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.7 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [ ]:
clinical_note = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

print (clinical_note)

clinical_note_df = spark.createDataFrame([[clinical_note]]).toDF("text")


A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20

In [ ]:
icd10_result = sbert_models.transform(clinical_note_df)

In [ ]:
pd.set_option("display.max_colwidth",0)

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)

def get_icd_codes(icd10_res):
    
    icd10_df = icd10_res.select(F.explode(F.arrays_zip('ner_chunk.result', 
                                                       'ner_chunk.metadata', 
                                                       'icd10cm_code.result', 
                                                       'icd10cm_code.metadata')).alias("cols")) \
                                 .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                         F.expr("cols['0']").alias("ner_chunk"),
                                         F.expr("cols['1']['entity']").alias("entity"), 
                                         F.expr("cols['2']").alias("icd10_code"),
                                         F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                         F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()


    
    codes = []
    resolutions = []

    for code, resolution in zip(icd10_df['all_codes'], icd10_df['resolutions']):

        codes.append(code.split(':::'))
        resolutions.append(resolution.split(':::'))

    icd10_df['all_codes'] = codes  
    icd10_df['resolutions'] = resolutions
    
    return icd10_df

In [ ]:
%%time

res_pd = get_icd_codes(icd10_result)
res_pd.head(15)

CPU times: user 57.6 ms, sys: 12 ms, total: 69.6 ms
Wall time: 3min 3s


,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions
0,0,gestational diabetes mellitus,PROBLEM,O2441,"[O2441, O2443, Z8632, Z875, O2431, O2411, O244, O241, O2481]","[gestational diabetes mellitus, postpartum gestational diabetes mellitus, history of gestational diabetes mellitus, history of gestational diabetes mellitus (situation), pre-existing diabetes mellitus in pregnancy, pre-existing diabetes mellitus in pregnancy (disorder), postpartum gestational diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, other pre-existing diabetes mellitus in pregnancy]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O2411,"[O2411, E118, E119, E139, E113, E1144, Z863, Z8639, E1132, E1143, E117, E115]","[pre-existing type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus, secondary diabetes mellitus, disorder of eye due to type 2 diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation), history of diabetes mellitus type 2, secondary endocrine diabetes mellitus, neurological disorder with type 2 diabetes mellitus, multiple complications of type 2 diabetes mellitus, peripheral circulatory disorder due to type 2 diabetes mellitus]"
2,0,T2DM,PROBLEM,E119,"[E119, E118, O2411, E109, E139, E113, E148, E149, E8881, Z833, E117, D649, Z863]","[type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, diabetes mellitus, secondary diabetes mellitus, disorder of eye with type 2 diabetes mellitus, disorder associated with diabetes mellitus, dm - diabetes mellitus, insulin resistance in diabetes, fh: diabetes mellitus, multiple complications of type 2 diabetes mellitus, anemia of diabetes, h/o: diabetes mellitus]"
3,0,HTG-induced pancreatitis,PROBLEM,K8520,"[K8520, K853, K8590, F102, K852, K859, K8580, K8591, K858, K918, K8510]","[alcohol-induced pancreatitis, drug-induced acute pancreatitis, hemorrhagic pancreatitis, alcohol-induced chronic pancreatitis, alcohol-induced acute pancreatitis (disorder), pancreatitis, traumatic acute pancreatitis (disorder), necrotizing pancreatitis, traumatic acute pancreatitis, post-ercp acute pancreatitis, gallstone acute pancreatitis]"
4,0,an acute hepatitis,PROBLEM,B159,"[B159, K720, B179, B172, Z0389, K752, K712, B199, B161, K701, B169]","[acute hepatitis a, acute hepatitis, acute infectious hepatitis, acute hepatitis e, acute infectious hepatitis suspected, acute focal hepatitis, toxic liver disease with acute hepatitis, fulminant hepatitis, acute hepatitis b, acute alcoholic hepatitis, acute fulminating viral hepatitis]"
5,0,obesity,PROBLEM,E669,"[E669, E668, Z6841, Q130, E66, E6601, Z8639, E349, H3550, Z8349, Q5562]","[obesity, abdominal obesity, obese, central obesity, overweight and obesity, morbid obesity, h/o: obesity, severe obesity, centripetal obesity, fh: obesity, truncal obesity]"
6,0,a body mass index,PROBLEM,Z6841,"[Z6841, E669, R229, Z681, R223, R221, Z68, R222, R220, R4189, M6258, P299, R1900, R898, M2170, R190, M2180]","[finding of body mass index, observation of body mass index, mass of body region, finding of body mass index (finding), mass of upper limb, head and neck mass, body mass index [bmi], mass of trunk, mass of head, preoccupation with body size, finding of size of upper limb, mass of cardiovascular structure, mass of abdominal cavity structure, increase in circumference, finding of proportion of upper limb, visible abdominal mass, observation of size of upper limb]"
7,0,polyuria,PROBLEM,R35,"[R35, R358, E232, R31, R350, R8299, N401, E723, O048, R300, R319, E7201, R809, E888, R808, N030, P960, R80, N026, B80]","[polyuria, polyuric state, polyuric state (disorder), hematuria, micturition frequency and polyuria, uricosuria, pollakiuria, saccharopinuria, oligouria, dysuria, hematuria syndrome, cystinuria, cyclic proteinuria, sialuria, paroxysmal proteinuria, renal hematuria (disorder), 

Lets apply some HTML formating by using `sparknlp_display` library to see the results of the pipeline in a nicer layout:

In [ ]:
from sparknlp_display import EntityResolverVisualizer

# with light pipeline
light_model = LightPipeline(sbert_models)

vis = EntityResolverVisualizer()

# Change color of an entity label
vis.set_label_colors({'PROBLEM':'#008080'})

light_data_icd = light_model.fullAnnotate(clinical_note)

vis.display(light_data_icd[0], 'ner_chunk', 'icd10cm_code')


## BertSentenceChunkEmbeddings

This annotator let users to aggregate sentence embeddings and ner chunk embeddings to get more specific and accurate resolution codes. It works by averaging context and chunk embeddings to get contextual information. Input to this annotator is the context (sentence) and ner chunks, while the output is embedding for each chunk that can be fed to the resolver model. The `setChunkWeight` parameter can be used to control the influence of surrounding context. 

For more information and examples of `BertSentenceChunkEmbeddings` annotator, you can check here: 
[24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb)

### ICD10CM with BertSentenceChunkEmbeddings

Lets do the same process by using `BertSentenceEmbeddings` annotator and compare the results. We will create a new pipeline by using this annotator with SentenceEntityResolverModel.

In [ ]:
#Get average sentence-chunk Bert embeddings
sentence_chunk_embeddings = BertSentenceChunkEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence", "ner_chunk"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)\
    .setChunkWeight(0.5) #default : 0.5


icd10_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")
  
resolver_pipeline_SCE = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        sentence_chunk_embeddings,
        icd10_resolver])


empty_data = spark.createDataFrame([['']]).toDF("text")
model_SCE = resolver_pipeline_SCE.fit(empty_data)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [ ]:
icd10_result_SCE = model_SCE.transform(clinical_note_df)

In [ ]:
%%time

res_SCE_pd = get_icd_codes(icd10_result_SCE)

res_SCE_pd.head(15)

CPU times: user 89.7 ms, sys: 5.52 ms, total: 95.2 ms
Wall time: 4min 29s


,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions
0,0,gestational diabetes mellitus,PROBLEM,O244,"[O244, O2443, O2441, O2442, O2413]","[gestational diabetes mellitus, class d, gestational diabetes mellitus in the puerperium, gestational diabetes mellitus, class a>1<, gestational diabetes mellitus in childbirth, pre-existing type 2 diabetes mellitus, in the puerperium]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O2413,"[O2413, E116, E1143, O2411, E1110, E1133, O241, E114, O2412, E111, E1169, E113, K318, E112, O244, H211X, E109, E1129, E162]","[pre-existing type 2 diabetes mellitus, in the puerperium, gastroparesis co-occurrent and due to type 2 diabetes mellitus, gastroparesis with type 2 diabetes mellitus, pregnancy and type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus, moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus in pregnancy, cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, in childbirth, ketoacidosis due to type 2 diabetes mellitus (disorder), angina due to type 2 diabetes mellitus, rubeosis iridis co-occurrent and due to type 2 diabetes mellitus, diabetic gastroparesis associated with type 2 diabetes mellitus, persistent microalbuminuria due to type 2 diabetes mellitus, gestational diabetes mellitus, class d, rubeosis iridis due to type 2 diabetes mellitus, acanthosis nigricans due to type 2 diabetes mellitus, persistent microalbuminuria due to type 2 diabetes mellitus (disorder), post hypoglycemic hyperglycemia due to diabetes mellitus]"
2,0,T2DM,PROBLEM,O244,"[O244, O2413, O2411, L83, O241, E1143, E119, E113, O2441, O2412, E116, E109, E1169, E1135, E1110]","[gestational diabetes mellitus, class a>2<, pre-existing type 2 diabetes mellitus, in the puerperium, pregnancy and type 2 diabetes mellitus, type 2 diabetes mellitus with acanthosis nigricans, pre-existing type 2 diabetes mellitus in pregnancy, gastroparesis with type 2 diabetes mellitus, type 2 diabetes mellitus in obese, rubeosis iridis with type 2 diabetes mellitus, gestational diabetes mellitus, class a>1<, pre-existing type 2 diabetes mellitus, in childbirth, gastroparesis co-occurrent and due to type 2 diabetes mellitus, acanthosis nigricans due to type 2 diabetes mellitus, diabetes mellitus type 2 in obese (disorder), type 2 diabetes with stable prolif diabetic rtnop, bilateral, ketoacidosis in type 2 diabetes mellitus]"
3,0,HTG-induced pancreatitis,PROBLEM,E139,"[E139, E1069, Z924, K903, K8532, E104, K918, K8502, K8522, K318, K8512, E1143, E46, K8510, E106, E091, K8590, E1043]","[congenital insulin-dependent diabetes mellitus with fatal secretory diarrhea, congenital insulin-dependent diabetes mellitus with fatal secretory diarrhoea, postprocedural pancreatic steatorrhea, postprocedural pancreatic steatorrhea (disorder), drug induced acute pancreatitis with infected necrosis, diabetes type 1 with gastroparesis, post-endoscopic retrograde cholangiopancreatography acute pancreatitis (disorder), idiopathic acute pancreatitis with infected necrosis, alcohol induced acute pancreatitis with infected necrosis, gastroparesis due to diabetes mellitus type i, biliary acute pancreatitis with infected necrosis, gastroparesis due to diabetes mellitus type ii, ketoacidosis due to malnutrition related diabetes mellitus (disorder), gallstone acute pancreatitis (disorder), gastroparesis due to type 1 diabetes mellitus, drug or chemical induced diabetes mellitus with ketoacidosis, acute hemorrhagic pancreatitis (disorder), gastroparesis with type 1 diabetes mellitus]"
4,0,an acute hepatitis,PROBLEM,B169,"[B169, B159, B162, B179, B160, K712, E8021, F1098, B181, G318, O9843, O266]","[acute fulminating type b viral hepatitis (disorder), acute fulminating type a viral hepatitis (disorder), hepatic coma due to acute hepatitis b, acute fulminating viral hepatitis (disorder), acute viral hepatitis b with hepa

In [ ]:
icd10_SCE_lp = LightPipeline(model_SCE)

light_result = icd10_SCE_lp.fullAnnotate(clinical_note)

visualiser = EntityResolverVisualizer()

# Change color of an entity label
visualiser.set_label_colors({'PROBLEM':'#008080'})

visualiser.display(light_result[0], 'ner_chunk', 'icd10cm_code')

**Lets compare the results that we got from these two methods.**

In [ ]:
sentence_df = icd10_result.select(F.explode(F.arrays_zip('sentence.metadata', 'sentence.result')).alias("cols")) \
                                .select( F.expr("cols['0']['sentence']").alias("sent_id"),
                                         F.expr("cols['1']").alias("sentence_all")).toPandas()

comparison_df = pd.merge(res_pd.loc[:,'sent_id':'resolutions'],res_SCE_pd.loc[:,'sent_id':'resolutions'], on=['sent_id',"ner_chunk", "entity"], how='inner')
comparison_df.columns=['sent_id','ner_chunk', 'entity', 'icd10_code', 'all_codes', 'resolutions', 'icd10_code_SCE', 'all_codes_SCE', 'resolutions_SCE']

comparison_df = pd.merge(sentence_df, comparison_df,on="sent_id").drop('sent_id', axis=1)
comparison_df.head(15)

,sentence_all,ner_chunk,entity,icd10_code,all_codes,resolutions,icd10_code_SCE,all_codes_SCE,resolutions_SCE
0,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",gestational diabetes mellitus,PROBLEM,O2441,"[O2441, O2443, Z8632, Z875, O2431, O2411, O244, O241, O2481]","[gestational diabetes mellitus, postpartum gestational diabetes mellitus, history of gestational diabetes mellitus, history of gestational diabetes mellitus (situation), pre-existing diabetes mellitus in pregnancy, pre-existing diabetes mellitus in pregnancy (disorder), postpartum gestational diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, other pre-existing diabetes mellitus in pregnancy]",O244,"[O244, O2443, O2441, O2442, O2413]","[gestational diabetes mellitus, class d, gestational diabetes mellitus in the puerperium, gestational diabetes mellitus, class a>1<, gestational diabetes mellitus in childbirth, pre-existing type 2 diabetes mellitus, in the puerperium]"
1,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",subsequent type two diabetes mellitus,PROBLEM,O2411,"[O2411, E118, E119, E139, E113, E1144, Z863, Z8639, E1132, E1143, E117, E115]","[pre-existing type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus, secondary diabetes mellitus, disorder of eye due to type 2 diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation), history of diabetes mellitus type 2, secondary endocrine diabetes mellitus, neurological disorder with type 2 diabetes mellitus, multiple complications of type 2 diabetes mellitus, peripheral circulatory disorder due to type 2 diabetes mellitus]",O2413,"[O2413, E116, E1143, O2411, E1110, E1133, O241, E114, O2412, E111, E1169, E113, K318, E112, O244, H211X, E109, E1129, E162]","[pre-existing type 2 diabetes mellitus, in the puerperium, gastroparesis co-occurrent and due to type 2 diabetes mellitus, gastroparesis with type 2 diabetes mellitus, pregnancy and type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus, moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus in pregnancy, cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, in childbirth, ketoacidosis due to type 2 diabetes mellitus (disorder), angina due to type 2 diabetes mellitus, rubeosis iridis co-occurrent and due to type 2 diabetes mellitus, diabetic gastroparesis associated with type 2 diabetes mellitus, persistent microalbuminuria due to type 2 diabetes mellitus, gestational diabetes mellitus, class d, rubeosis iridis due to type 2 diabetes mellitus, acanthosis nigricans due to type 2 diabetes mellitus, persistent microalbuminuria due to type 2 diabetes mellitus (disorder), post hypoglycemic hyperglycemia due to diabetes mellitus]"
2,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BM